# Getting Structured Outputs from Llama3 using Ollama 

In [ ]:
!pip install -U ollama pydantic

In [1]:
from ollama import chat
from pydantic import BaseModel
from typing import Optional, List

class Task(BaseModel):
    title: str
    description: str
    priority: str  # "High", "Medium", "Low"
    estimated_hours: float
    dependencies: List[str] = []
    status: str = "Not Started"  # "Not Started", "In Progress", "Completed"
    assigned_to: Optional[str] = None

response = chat(
    messages=[
        {
            'role': 'user',
            'content': 'Create a task for implementing a new feature in our project management software that allows users to track time spent on tasks. \
            Include dependencies and make it realistic.',
        }
    ],
    model='llama3.1',
    format=Task.model_json_schema(),
)

task = Task.model_validate_json(response['message']['content'])
task

Task(title='Implement Time Tracking Feature', description='Allow users to log time spent on tasks, with features to pause and resume tracking.', priority='High', estimated_hours=120.0, dependencies=['Database schema update ( #123 )', 'UX redesign ( #456 )'], status='In Progress', assigned_to=None)

In [2]:
print(f"Task: {task.title}")
print(f"Priority: {task.priority}")
print(f"Estimated Hours: {task.estimated_hours}")
print(f"Dependencies: {', '.join(task.dependencies)}")
print(f"Status: {task.status}")

Task: Implement Time Tracking Feature
Priority: High
Estimated Hours: 120.0
Dependencies: Database schema update ( #123 ), UX redesign ( #456 )
Status: In Progress


In [8]:
class EmailToolInput(BaseModel):
    email_destination: str
    email_contents: str


prompt = """
Write an email to my boss: boss_of_lucas@gmail.com, telling him that I quit to pursue bird watching.
"""

email_reply_input = chat(
    model='llama3.2',
    messages=[
        {
            'role': 'user',
            'content': prompt,
        }
    ],
    format=EmailToolInput.model_json_schema(),
)

email_reply_input

{'model': 'llama3.2',
 'created_at': '2025-03-12T17:56:27.599121Z',
 'message': {'role': 'assistant',
  'content': '{ "email_destination": "boss_of_lucas@gmail.com", "email_contents": "<h1>Bird Watching and a New Chapter</h1>Dear {Boss}\'s Name,\\r\\n\\r\\nI am writing to inform you of my decision to resign from my position as [Position] effective [Date]. This was not an easy decision for me, but I have decided that it is time for me to pursue a new passion – bird watching.\\r\\n\\r\\nAs many know, birds have always held a special fascination for me. Over the years, I\'ve had the privilege of observing and learning about various species, and my interest has only grown stronger with each passing day. I feel an overwhelming urge to dedicate myself to this hobby, to explore new territories, and to contribute to the understanding and conservation of bird populations.\\r\\n\\r\\nI want to express my gratitude for the opportunities I\'ve had while working here. I have learned a great deal, b

In [9]:
email_reply_input_json = EmailToolInput.model_validate_json(email_reply_input['message']['content'])
print(email_reply_input_json.email_destination)

print(email_reply_input_json.email_contents)

boss_of_lucas@gmail.com
<h1>Bird Watching and a New Chapter</h1>Dear {Boss}'s Name,

I am writing to inform you of my decision to resign from my position as [Position] effective [Date]. This was not an easy decision for me, but I have decided that it is time for me to pursue a new passion – bird watching.

As many know, birds have always held a special fascination for me. Over the years, I've had the privilege of observing and learning about various species, and my interest has only grown stronger with each passing day. I feel an overwhelming urge to dedicate myself to this hobby, to explore new territories, and to contribute to the understanding and conservation of bird populations.

I want to express my gratitude for the opportunities I've had while working here. I have learned a great deal, both personally and professionally, and I appreciate the experience and connections I've made during my tenure.

However, I feel that it is essential for me to follow my heart and pursue this new

In [10]:
def send_email(email_destination: str, email_contents: str):
    print("Pretending I am sending an email")
    print(f"Sending email to {email_destination} with contents: {email_contents}")

send_email(email_reply_input_json.email_destination, email_reply_input_json.email_contents)

Pretending I am sending an email
Sending email to boss_of_lucas@gmail.com with contents: <h1>Bird Watching and a New Chapter</h1>Dear {Boss}'s Name,

I am writing to inform you of my decision to resign from my position as [Position] effective [Date]. This was not an easy decision for me, but I have decided that it is time for me to pursue a new passion – bird watching.

As many know, birds have always held a special fascination for me. Over the years, I've had the privilege of observing and learning about various species, and my interest has only grown stronger with each passing day. I feel an overwhelming urge to dedicate myself to this hobby, to explore new territories, and to contribute to the understanding and conservation of bird populations.

I want to express my gratitude for the opportunities I've had while working here. I have learned a great deal, both personally and professionally, and I appreciate the experience and connections I've made during my tenure.

However, I feel t

We can turn this structured output functionality into a function calling pipeline.